In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/MyDrive/baselines/muss'
!pip install -e .

In [ ]:
import os
import numpy as np
import ast
import matplotlib.pyplot as plt

import re
import math
import pickle
from itertools import product 
import copy 

import pandas as pd
import csv
!pip install google.colab

from google.colab import drive
drive.mount('/content/gdrive') 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.2 MB/s 
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import argparse 

from muss.simplify import ALLOWED_MODEL_NAMES, simplify_sentences


In [ ]:
!python scripts/simplify.py scripts/examples.en --model-name muss_en_wikilarge_mined


tcmalloc: large alloc 1625161728 bytes == 0x7624a000 @  0x7f3739866b6b 0x7f3739886379 0x7f36a6401d57 0x7f36a63efbc3 0x7f36f81fc1f1 0x5946b8 0x548cc1 0x51566f 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x549576 0x593fce 0x5118f8 0x549576 0x593fce
tcmalloc: large alloc 1625161728 bytes == 0xd702a000 @  0x7f3739866b6b 0x7f3739886379 0x7f36a6401d57 0x7f36a63efbc3 0x7f36f81fc1f1 0x5946b8 0x548cc1 0x51566f 0x549e0e 0x4bcb19 0x59582d 0x595b69 0x62026d 0x55de15 0x59af67 0x515655 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x549576 0x593fce 0x5118f8 0x549576 0x593fce
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This result

In [ ]:
%cd /content/gdrive/MyDrive/baselines/muss

/content/gdrive/MyDrive/baselines/muss


In [ ]:
from muss.simplify import simplify_sentences


In [ ]:
simplify_sentences(['Brunstad has several fast food restaurants , a cafeteria-style restaurant , coffee bar , and its own grocery store.'])

['Brunstad has a cafeteria-style restaurant, several fast food restaurants, a coffee bar, and a grocery store.']

In [ ]:
from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset

from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

In [ ]:
import pickle

In [ ]:
# with open('/content/gdrive/MyDrive/baselines/muss/qualitative/0913_ABCD_NER_wikilarge', 'rb') as fp:
#   test_df = pickle.load(fp)

In [ ]:
# generating the output 
%cd '/content/gdrive/MyDrive/baselines/muss'


from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name
from easse.utils.helpers import read_lines





/content/gdrive/MyDrive/baselines/muss


In [ ]:
output_dir = '/content/gdrive/MyDrive/baselines/muss/output/'

In [ ]:
def generate_output(chosen_id_list,data_dir_list,data_list,sample=False,add_to_dir_name=None,muss_output=False,NE_output=False,CERF_output=False,NE_CERF_output=False,hypothesis_num=None,beam=None,sampling=False):

  # store created dir
  created_dir = []

  for index in chosen_id_list:
    
    # get model unique local id
    exp_dir = model_dir_dict[index]['exp_dir']
    print('| model_name is',model_dir_dict[index]['model_name'])
    print('| exp_dir is',exp_dir)
    model_name = 'model_'+ str(index) + '_'+ exp_dir.split('/')[-2]

    os.makedirs(output_dir+model_name, exist_ok=True)

    language = 'en'

    # get recommended_preprocessors_kwargs
    if 'recommended_preprocessors_kwargs' in model_dir_dict[index].keys() :
      preprocessors_kwargs = model_dir_dict[index]['recommended_preprocessors_kwargs']
      print('| use provided kwargs')
    else:
      # use muss_mined_wiki preprocessors
      preprocessors_kwargs = {
            'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False},
            'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False},
            'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': language, 'use_short_name': False},
            'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': language, 'use_short_name': False},
        }
      preprocessors_kwargs['GPT2BPEPreprocessor'] = {}

    print('| use preprocessors_kwargs',preprocessors_kwargs)
    preprocessors = get_preprocessors(preprocessors_kwargs)

    if hypothesis_num == None:
      hypothesis_num = 1
    if beam == None:
      beam=5

    generate_kwargs = {'hypothesis_num':hypothesis_num,
                       'beam':beam,
                       'sampling':sampling}

    simplifier = get_fairseq_simplifier(exp_dir, **generate_kwargs)
    simplifier = get_preprocessed_simplifier(simplifier, preprocessors=preprocessors)

    #########################################
    # simplify sample sentences
    if sample:
      if muss_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_no_token.en'
      elif NE_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_token.en'
      elif CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_ABCD_token.en'
      elif NE_CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_ABCD_token.en'

      pred_path = simplifier(complex_file_dir)


      for i in range(len(read_lines(complex_file_dir))):
        print('original:\n',read_lines(complex_file_dir)[i])
        print('simplified:\n',read_lines(pred_path)[i])
        print('----------------------------------------------------')

      return read_lines(complex_file_dir),read_lines(pred_path)
    ###########################################

    # create dir for output
    subfolders = os.listdir(output_dir + model_name)
    num_subfolder = len(subfolders)
    new_folder_dir = output_dir + model_name + '/' +str(num_subfolder).zfill(2) +'/'
    os.mkdir(new_folder_dir)
    created_dir.append(new_folder_dir)

     # simplify asset dataset
    for index, file_dir in enumerate(data_dir_list):
      
      info_file_name = new_folder_dir + 'info.txt'
      with open(info_file_name, 'w') as f:
        f.write("generate_kwargs %s\n" % generate_kwargs)
      f.close()

      filename = new_folder_dir + data_list[index]
      pred_path = simplifier(file_dir)

      with open(filename, 'w') as f:
          for item in read_lines(pred_path):
              f.write("%s\n" % item)

  print('created_dir',created_dir)
  return created_dir

In [ ]:
# wikilarge detokenized baseline
test_data_dir_list,test_data_list = ['/content/gdrive/MyDrive/baselines/muss/resources/datasets/wikilarge_detokenized/test.complex'],['valid.diff.complex.pred']


In [ ]:
test_data_dir_list

['/content/gdrive/MyDrive/baselines/muss/resources/datasets/wikilarge_detokenized/test.complex']

In [ ]:
# wikilarge with complex and next_ne prefixes

test_data_dir_list,test_data_list = ['/content/gdrive/MyDrive/baselines/muss/resources/datasets/asset/valid.complex'],['valid.complex.pred']
test_data_dir_list.append('/content/gdrive/MyDrive/baselines/muss/resources/datasets/0913_ABCD_NER_wikilarge/valid.complex')
test_data_list.append('valid.diff.complex.pred')

In [ ]:
test_data_dir_list

['/content/gdrive/MyDrive/baselines/muss/resources/datasets/asset/valid.complex',
 '/content/gdrive/MyDrive/baselines/muss/resources/datasets/0913_ABCD_NER_wikilarge/valid.complex']

In [ ]:
def show_simplify(complex_path,simple_path = None):
  count = 0 
  if simple_path != None:
    print('simplified:\n')
    temp_csv_simp = []
    temp_csv_comp = []
    for i in range(len(read_lines(simple_path))):
      if count <100:
        print('-'*80,'\n','complex:',read_lines(complex_path)[i])
        print('simple:',read_lines(simple_path)[i])
        count += 1 
      temp_csv_simp.append(read_lines(simple_path)[i])
      temp_csv_comp.append(read_lines(complex_path)[i])
    return temp_csv_comp, temp_csv_simp
  else:
    temp_csv_comp = []
    for i in range(len(read_lines(complex_path))):
      # print('-'*80,'\n','complex:',read_lines(complex_path)[i])
      temp_csv_comp.append(read_lines(complex_path)[i])
    return temp_csv_comp

In [ ]:
model_dir_dict = {}

def add_item_to_dict(**kwargs):

  id = len(model_dir_dict)
  model_dir_dict[id]={}

  for key, value in kwargs.items():
    model_dir_dict[id][key]=value
  print('added:',model_dir_dict[id])


In [ ]:
# This dataset should exist in resources/datasets/ and contain the following files:
# train.complex, train.simple, valid.complex, valid.simple, test.complex, test.simple
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')


def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'

    bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/bart_mined/model.pt'
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 1,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 3e-05,
            'warmup_updates': 500, #500
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 20000, # 20000
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': get_evaluate_kwargs(language),
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

In [ ]:
exp_dir ='/content/gdrive/MyDrive/baselines/muss/experiments/fairseq/wikilarge_prefix/'

In [ ]:
dataset = '0913_ABCD_NER_wikilarge'
kwargs = get_bart_kwargs(dataset=dataset, language='en', use_access=True)
kwargs['train_kwargs']['ngpus'] = 1  # Set this from 8 to 1 for local training
kwargs['train_kwargs']['max_tokens'] = 512  # Lower this number to prevent OOM

from muss.fairseq.main import find_best_parametrization
recommended_preprocessors_kwargs = find_best_parametrization(exp_dir, **kwargs)
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')
kwargs['preprocessor_kwargs'] = recommended_preprocessors_kwargs

/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
[nltk_data] Downloading package punkt to 

recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}


In [ ]:
#@title
import time
while True:
  time.sleep(0)

KeyboardInterrupt: ignored

In [ ]:
# recommended preprocessor kwargs for prefix muss model
recommended_preprocessors_kwargs 

{'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744,
  'use_short_name': False},
 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889,
  'use_short_name': False},
 'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245,
  'language': 'en',
  'use_short_name': False},
 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092,
  'language': 'en',
  'use_short_name': False},
 'GPT2BPEPreprocessor': {}}

In [ ]:
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')

recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}


In [ ]:
MODEL_DIR = '/content/gdrive/MyDrive/baselines/muss/experiments/fairseq/'

In [ ]:
# muss trained on wikilarge with prefixes (updated)
add_item_to_dict(model_id=len(model_dir_dict),exp_dir=MODEL_DIR+'local_1661818681253/',model_name='wikilarge_prefix',
                 recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
)

added: {'model_id': 0, 'exp_dir': '/content/gdrive/MyDrive/baselines/muss/experiments/fairseq/local_1661818681253/', 'model_name': 'wikilarge_prefix', 'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}}


In [ ]:
model_dir_dict

{0: {'model_id': 0,
  'exp_dir': '/content/gdrive/MyDrive/baselines/muss/experiments/fairseq/local_1661818681253/',
  'model_name': 'wikilarge_prefix',
  'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 1.4836625994901744,
    'use_short_name': False},
   'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7562703621541889,
    'use_short_name': False},
   'WordRankRatioPreprocessor': {'target_ratio': 0.8249666574125245,
    'language': 'en',
    'use_short_name': False},
   'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.1273178104478092,
    'language': 'en',
    'use_short_name': False},
   'GPT2BPEPreprocessor': {}}}}

In [ ]:
# path of the model for wikilarge_prefix:  /content/gdrive/MyDrive/baselines/muss/experiments/fairseq/wikilarge_prefix

# simplify_sentences(['this is a regular sentence'], model_name='')

# !python scripts/simplify.py /content/gdrive/MyDrive/baselines/muss/qualitative/test_df_complex_sentence --model-name wikilarge_prefix
!python scripts/simplify.py /content/gdrive/MyDrive/baselines/resources/datasets/asset --model-name muss_en_wikilarge_mined


Traceback (most recent call last):
  File "scripts/simplify.py", line 24, in <module>
    source_sentences = read_lines(args.filepath)
  File "/content/gdrive/MyDrive/baselines/muss/muss/utils/helpers.py", line 156, in read_lines
    return list(yield_lines(filepath, gzipped=gzipped))
  File "/content/gdrive/MyDrive/baselines/muss/muss/utils/helpers.py", line 148, in yield_lines
    with open_function(filepath, 'rt', encoding='utf-8') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/baselines/resources/datasets/asset'


In [ ]:
# local_1661818681253 wikilarge_prefix 

token_ori_prefix = generate_output([0],test_data_dir_list,test_data_list)

In [ ]:
# findfint the ACCESS parameters for the baseline muss  

dataset = 'wikilarge_detokenized'
kwargs = get_bart_kwargs(dataset=dataset, language='en', use_access=True)
kwargs['train_kwargs']['ngpus'] = 1  # Set this from 8 to 1 for local training
kwargs['train_kwargs']['max_tokens'] = 512  # Lower this number to prevent OOM

from muss.fairseq.main import find_best_parametrization
recommended_preprocessors_kwargs = find_best_parametrization(exp_dir, **kwargs)
print(f'recommended_preprocessors_kwargs={recommended_preprocessors_kwargs}')
kwargs['preprocessor_kwargs'] = recommended_preprocessors_kwargs

In [ ]:
# baseline muss trained on wikilarge_detokenized (yet to be updated)


add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1660919962534/',model_name='local_',
                 recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 1.4874629760208349, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.7234891196330869, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 0.8666465851166344, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.6345124065030069, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}})

In [ ]:
model_dir_dict

In [ ]:
token_ori_2 = generate_output([0],test_data_dir_list,test_data_list)